In [3]:
import json
import pickle as pkl
import os 

In [4]:
with open('test.story-in-sequence.json', 'rb') as f_test_sis:
    test_sis = json.load(f_test_sis)
    print(type(test_sis))
    print(len(test_sis['annotations']))

<class 'dict'>
25275


In [5]:
with open('test.storyid2photos_list.json', 'rb') as s2p:
    s2p_dict = json.load(s2p)

#inspect here
for storyid in s2p_dict:
    target = s2p_dict[storyid]
    
    for entry in s2p_dict:
        if target == s2p_dict: 
            print("for", storyid)
            print(entry, "is duplicate") # no duplicates in testset photostream!

In [47]:
25275/5


5055.0

In [6]:
for entity in test_sis['annotations'][:10]:
    print(entity[0]['worker_arranged_photo_order'], entity[0]['story_id'], ":", entity[0]['text'])

0 45530 : the local parish holds a craft show each year .
1 45530 : lots of folks come out and set up tables to sell their crafts .
2 45530 : some of these crafts are very unique and take a lot of talent to make .
3 45530 : folks of all ages come out to peruse the crafts for sale .
4 45530 : some of the crafters even dress up in unique costumes as part of their selling act .
0 45531 : i was so excited to be heading to the crafts fair .
1 45531 : when i arrived i saw a great booth with a variety of great crafts .
2 45531 : i stopped at chatted at my friend [female] 's booth for a bit .
3 45531 : there were even booths set up for all of the kids .
4 45531 : i found some awesome crafts at the fair , i 'm really happy that i went .


In [7]:
test_sis['annotations'][29*5][0]['original_text']

'We had a great view of Minute Maid Park from our hotel room.'

In [10]:
test_sis['annotations'][31][0]['worker_arranged_photo_order']

1

In [11]:
def storyID2norm_txt(test_sis_annot):
    storyID2norm_txt = {}
    storyID = test_sis_annot[0][0]['story_id'] #init storyID
    sen_list = []
    for entity in test_sis_annot:
        if storyID != entity[0]['story_id']:
            #add to dict
            norm_txt = " ".join(sen_list) # concat sentences in the list with whitespace as a delim
            storyID2norm_txt[storyID] =  norm_txt
            #renew sid, sen_list
            storyID = entity[0]['story_id'] 
            sen_list = [] 
        #general behavior for gathering sentences 
        sentence = entity[0]['text']    
        sen_list.append(sentence) 
    return storyID2norm_txt
        

In [12]:
def what_storyID(ps, s2p_dict): # working good 
    index = list(s2p_dict.values()).index(ps) # if there is no ps in the values, it raises error "ps is not in the list"
    story_id = list(s2p_dict.keys())[index] 
    
    return story_id

In [13]:
s2nt_dict = storyID2norm_txt(test_sis['annotations'])

In [14]:
with open("new_test.storyid2txt.json", "w") as f: #json dump supports str so you need to use w rather than wb 
    json.dump(s2nt_dict,f)

In [15]:
def htmls2ps(htmldir):
    ps_list = []
    for html_name in os.listdir(htmldir):
        splited = html_name.split('-')
        ps = splited[:5]
        ps_list.append(ps)
    return ps_list


def get_storyid_from_htmls(htmldir,s2p_dict):
    html_sid_list = []
    ps_list = htmls2ps(htmldir)
    for ps in ps_list:
        sid = what_storyID(ps, s2p_dict)
        html_sid_list.append(sid)
    return html_sid_list
    
    

In [16]:
htmldir = "/home/seonils/data_storage/vist200web/www"
html_storyids = get_storyid_from_htmls(htmldir,s2p_dict)

In [17]:
def html_sids2ps_n_nt(html_storyids, s2p_dict, s2nt_dict):
    html_sids2ps_dict = {}
    html_sids2nt_dict = {} 
    for sid in html_storyids:
        html_sids2ps_dict[sid] = s2p_dict[sid]
        html_sids2nt_dict[sid] = s2nt_dict[sid]
    return html_sids2ps_dict, html_sids2nt_dict 

In [18]:
h_sid2ps, h_sid2nt = html_sids2ps_n_nt(html_storyids, s2p_dict, s2nt_dict)

In [19]:
with open("200html_sid2photostream.json", "w")  as ps_f, open("200html_sid2norm_txt.json", "w") as nt_f:
    json.dump(h_sid2ps, ps_f)
    json.dump(h_sid2nt, nt_f)

In [48]:
with open("SnuBiVtt1_298.json", "r") as f:
    glacnet_dict = json.load(f)
    #print(type(model_out))
    #print(model_out.keys())

In [49]:
glacnet_dict['output_stories'][0].keys()

dict_keys(['photo_sequence', 'album_id', 'story_text_normalized'])

In [50]:
def modelout_sid2normtxt(html_storyids, modelout_dict, h_sid2ps):
    modelout_sid2normtxt = {}
    for sid in html_storyids:
        ps = h_sid2ps[sid]
        for entry in modelout_dict['output_stories']:
            if ps == entry['photo_sequence']:
                modelout_sid2normtxt[sid] = entry['story_text_normalized']
    return modelout_sid2normtxt

In [53]:
glacnet_normtxt = modelout_sid2normtxt(html_storyids, glacnet_dict, h_sid2ps)

In [55]:
len(glacnet_normtxt)

200

In [57]:
with open("glacnet_200_normtxt.json", "w") as f:
    json.dump(glacnet_normtxt,f)

In [58]:
glacnet_keyslist = list(glacnet_normtxt.keys())

In [59]:
len(glacnet_keyslist)

200

In [60]:
keysplits = [glacnet_keyslist[20*i:20*(i+1)] for i in range(10)]


In [75]:
glac_dicts10_list = []
for keys in keysplits:
    stories = [glacnet_normtxt[key] for key in keys]
    glac_dicts10_list.append(dict(zip(keys,stories)))

#glac_dicts10_idx = dict(zip(range(10), glac_dicts10_list))
#print(glac_dicts10_idx.keys())
for i in range(10):
    with open("./glacnet_splits/glac_normtxt_spl_{}.json".format(i), "w") as f:
        json.dump(glac_dicts10_list[i], f)


In [71]:
type(list(glac_dicts10_idx.keys())[0])

int